# Adaptive Retrieval-Augmented Generation RAG

This notebook introduces an adaptive Retrieval-Augmented Generation (RAG) system that tailors its document retrieval strategy based on the type of query. The approach integrates advanced language model prompting to classify queries and then dynamically selects the appropriate retrieval strategy, resulting in more accurate and context-aware responses.

## System Introduction

The system is designed to overcome the limitations of traditional one-size-fits-all RAG approaches by adapting its retrieval method to the query’s nature. It uses a series of LLM-powered prompts to first classify the query, then to generate enhanced or reformulated queries, and finally to rank and select the most relevant documents. This adaptive process aims to yield results that are not only precise but also rich in context, making it suitable for a diverse range of query types.

## Underlying Concept

Traditional RAG systems often retrieve documents in a generic manner without considering the specific needs of different query types. In contrast, this system starts by classifying a user’s query into one of four categories: 

- **Factual:** For queries seeking verifiable and specific information.
- **Analytical:** For queries that require comprehensive analysis or explanation.
- **Opinion:** For queries that involve subjective perspectives or diverse viewpoints.
- **Contextual:** For queries that depend on user-specific context.

By differentiating between these types, the system adapts its retrieval strategy to enhance query precision, generate sub-questions when necessary, and integrate user context. This leads to richer, more coherent results tailored to the query’s requirements.

## System Components

1. **Query Classification Module:**
   - Uses a templated prompt (`ADAPTIVE_QUERY_CLASSIFIER_PROMPT`) to classify the incoming query into one of the four categories.
   - Determines if additional context is required when the query is contextual.

2. **Adaptive Retrieval Strategies:**
   - **Factual Strategy:** Enhances the query for precision, retrieves documents, and uses LLM-based ranking to select the top results.
   - **Analytical Strategy:** Generates multiple sub-queries for comprehensive coverage and applies diversity selection to ensure a broad analysis.
   - **Opinion Strategy:** Identifies distinct viewpoints and retrieves corresponding documents, then ranks them to cover a diverse range of opinions.
   - **Contextual Strategy:** Incorporates user-specific context to reformulate the query and ranks documents by considering both relevance and context.

3. **LLM-Enhanced Ranking:**
   - Each retrieval strategy uses specialized ranking prompts (e.g., `ADAPTIVE_FACTUAL_RANK_PROMPT`, `ADAPTIVE_CONTEXTUAL_RANK_PROMPT`) to evaluate document relevance on a scale, ensuring that the most pertinent documents are selected.

4. **Response Generation Module:**
   - Combines the selected documents into a final context and passes them, along with the original query, to an LLM (via `ADAPTIVE_FINAL_ANSWER_PROMPT`) to generate the final answer.

## Database Implementation and Llama Index Integration

The adaptive RAG system leverages the **Llama Index** as its underlying database for document ingestion, storage, and retrieval. This integration is evident in the use of the `retrieve_documents` function imported from `tools.rag.llama_index.retrieve`, which loads and queries a persistent vector store maintained by Llama Index.

By relying on Llama Index, the system benefits from automated document ingestion and vector indexing. Documents are processed using components like the `SimpleDirectoryReader` and, in other contexts, enriched with context windows using parsers like `SentenceWindowNodeParser`. This setup allows for efficient similarity search and ensures that the adaptive retrieval strategies (Factual, Analytical, Opinion, and Contextual) operate on a robust, scalable database.

For further details on how documents are ingested and indexed, please refer to the Llama Index ingestion section. This section provides comprehensive guidance on configuring the ingestion pipeline and customizing parameters to suit your specific data and retrieval requirements.

## How It Works

### 1. Query Classification

- The system starts by classifying the query using a prompt-based classifier. 
- If the query is classified as Contextual, any extracted user-specific context is captured and utilized in subsequent steps.

### 2. Adaptive Retrieval Strategies

- **Factual:** Enhances the query, retrieves documents, and then uses a ranking prompt to score each document's relevance.
- **Analytical:** Generates multiple sub-questions to cover different aspects, retrieves documents for each, and applies a diversity prompt to select a varied set of documents.
- **Opinion:** Extracts diverse viewpoints from the query, retrieves documents reflecting each viewpoint, and then ranks them to ensure representative opinions.
- **Contextual:** Reformulates the query by incorporating user context, retrieves documents, and ranks them based on both relevance and contextual alignment.

### 3. LLM-Enhanced Ranking

- For each strategy, a dedicated ranking prompt is used to assign a score to retrieved documents, ensuring the most relevant results are prioritized.

### 4. Response Generation

- The top-ranked documents are then aggregated and passed into a final prompt, which instructs an LLM to generate the complete answer based on the enriched context.

## Workflow Diagram

![Adaptive RAG Workflow](./adaptive_rag_workflow.png)

## System Advantages

- **Improved Accuracy:** Tailors retrieval methods to the specific query type, reducing irrelevant or incomplete results.
- **Flexibility:** Adapts to various query types (factual, analytical, opinion, contextual) ensuring that each is handled in the most effective manner.
- **Context-Awareness:** Incorporates user-specific context when needed, making the system capable of generating personalized responses.
- **Diverse Perspectives:** Actively retrieves and ranks documents to present a wide range of viewpoints for opinion-based queries.
- **Comprehensive Analysis:** For analytical queries, the system generates sub-queries that cover different aspects of the question for a thorough analysis.

## Practical Benefits

- **More Coherent Results:** The system’s ability to enhance and reformulate queries leads to outputs that are contextually richer and more understandable.
- **Reduced Fragmentation:** By using adaptive strategies, the approach avoids returning isolated or incomplete text fragments.
- **Customizable Retrieval:** The strategies can be fine-tuned (e.g., adjusting the number of sub-queries or the context window) to suit different datasets and user needs.
- **Enhanced Relevance:** LLM-powered ranking ensures that only the most relevant documents are selected, improving the quality of the final answer.

## Implementation Insights

- **retrieve.py:** Implements the core retrieval logic with various adaptive strategies (Factual, Analytical, Opinion, Contextual) and LLM-based ranking using multiple prompts.
- **prompts.py:** Contains all the prompt templates used by the system to classify queries, enhance or reformulate them, generate sub-queries, and rank retrieved documents.

## Parameters

**ADAPTIVE_RAG_MODEL:**
This environment variable specifies the language model used throughout the adaptive RAG system. It is critical for query classification, document ranking, and final answer generation, ensuring that the system utilizes the appropriate model for each task.

**ADAPTIVE_RAG_QUERY_TOP_K:**
This variable sets the number of top documents (top_k) to be considered during the retrieval process. It influences the breadth of the candidate documents that are ranked and ultimately selected for generating the final answer.

### Example Prompts

1. **Factual Version**
   - **Query:**
     "What specific metrics and methodologies do scientists use to evaluate GPT 4.5's completions?"
   - **Focus:** This version seeks clear, verifiable details such as evaluation metrics, experimental setups, or quantitative methods.

2. **Analytical Version**
   - **Query:**
     "How do scientists integrate quantitative metrics and qualitative assessments to comprehensively evaluate GPT 4.5's completions?"
   - **Focus:** This version prompts an in-depth analysis of the evaluation process, exploring the interplay between different evaluation methods and their implications.

3. **Opinion Version**
   - **Query:**
     "What are the various perspectives among experts regarding the effectiveness and fairness of current evaluation techniques for GPT 4.5's completions?"
   - **Focus:** This version targets subjective viewpoints and debates among scientists, inviting a discussion on the strengths and limitations of the evaluation methods.

4. **Contextual Version**
   - **Query:**
     "Considering the context of advancements in natural language processing and my background in AI research, how do scientists adapt their evaluation strategies for GPT 4.5's completions?"
   - **Focus:** This version incorporates user-specific or situational context, encouraging answers that account for recent trends, personal expertise, or specific research contexts.

## Conclusion

The Adaptive Retrieval-Augmented Generation (RAG) system presents a significant advancement in the field of information retrieval by tailoring its strategies based on the nature of the query. Through a systematic process of query classification, adaptive retrieval, and LLM-enhanced ranking, the system effectively generates context-aware and precise responses. Its ability to dynamically adjust retrieval methods ensures that the results are not only accurate but also rich in context, thereby addressing the limitations of traditional RAG approaches. Overall, this adaptive framework exemplifies a robust and scalable solution for handling diverse and complex queries.